# THAI data analysis (work in progress)

In [ ]:
# Standard library
from pathlib import Path

# Data analysis libraries
import dask.array as da
import xarray as xr

# Local modules
from calc import hdiv, moist_static_energy
from grid import reverse_along_dim, meridional_mean, zonal_mean
from model_um import adjust_um_grid, calc_um_ocean_frac, open_mf_um, prep_um_ds
from model_exocam import adjust_exocam_grid, calc_pres_exocam, calc_alt_exocam

Define a common THAI directory

In [ ]:
thai_dir = Path.cwd().parent / "data"

Create a dicitonary with model metadata

In [ ]:
models = {
    "LMDG": {
        "dir": thai_dir / "LMD-G",
        "chunks": {"Time": 61},
        "time": "Time",
        "lon": "longitude",
        "lat": "latitude",
        "lev": "altitude",
    },
    "UM": {
        "dir": thai_dir / "UM",
        "chunks": {"hourly": 61},
        "time": "hourly",
        "lon": "longitude_t",
        "lat": "latitude_t",
        "lev": "thlev_zsea_theta",
    },
    "ExoCAM": {
        "dir": thai_dir / "ExoCAM",
        "chunks": {"time": 61},
        "time": "time",
        "lon": "lon",
        "lat": "lat",
    },
}

In [ ]:
# # Contants
# from aeolus.const import init_const

# c = init_const("trap1e")

# r_planet = c.radius.data
# gas_const = c.dry_air_gas_constant.data
# c_p = c.dry_air_spec_heat_press.data
# g = c.gravity.data
# l_v = c.condensible_heat_vaporization.data

## Loading the data

Load all data

In [ ]:
# Load data
datasets = {}  # Create an empty dictionary to store all data
# Loop over THAI cases (this loop can swapped places with the loop below if needed)
for THAI_case in ["Hab1"]:  # , "Hab1star"]:
    # for each of the THAI cases, create a nested directory for models
    datasets[THAI_case] = {}
    for model_key, model_dict in models.items():
        if model_key == "UM":
            files = sorted(
                (models[model_key]["dir"] / THAI_case).glob(
                    f"{THAI_case}_{model_key}*.nc"
                )
            )
            ds = open_mf_um(
                files,
                main_time="hourly",
                rad_time="hourly_rad",
                decode_times=False,
                chunks=model_dict["chunks"],
            )
            ds = prep_um_ds(ds, vert_lev_miss_val="drop")
        else:
            ds = xr.open_mfdataset(
                str(model_dict["dir"] / THAI_case / f"{THAI_case}_{model_key}*.nc"),
                decode_times=False,
                combine="nested",
                concat_dim=model_dict["time"],
                chunks=model_dict["chunks"],
            )
            if model_key == "ExoCAM":
                _tmp_ds = {}
                for d in ds.data_vars:
                    _tmp_ds[d] = adjust_exocam_grid(ds[d])
                ds = xr.Dataset(_tmp_ds)
        datasets[THAI_case][model_key] = ds
del ds

## Diagnostics and plots

In [ ]:
# Prepare variables
model_key = "UM"
temp = datasets[THAI_case][model_key].STASH_m01s16i004.isel(
    **{models[model_key]["time"]: -1}
)  # .mean(dim=models["UM"]["time"]).compute()
u = datasets[THAI_case][model_key].STASH_m01s00i002.isel(
    **{models[model_key]["time"]: -1}
)
v = datasets[THAI_case][model_key].STASH_m01s00i003.isel(
    **{models[model_key]["time"]: -1}
)
pres = datasets[THAI_case][model_key].STASH_m01s00i408.isel(
    **{models[model_key]["time"]: -1}
)
spec_hum = datasets[THAI_case][model_key].STASH_m01s00i010.isel(
    **{models[model_key]["time"]: -1}
)
alt = datasets[THAI_case][model_key].thlev_zsea_theta
rho = pres / (gas_const * temp)

In [ ]:
# test_dse = vert_mer_mean_of_dse_flux(
#     temp,
#     alt,
#     spec_hum,
#     u,
#     v,
#     rho=rho,
#     zcoord_type="height",
#     lon_name=models[model_key]["lon"],
#     lat_name=models[model_key]["lat"],
#     z_name=models[model_key]["lev"],
#     c_p=c_p,
#     gravity=g,
#     latent_heat=l_v,
#     r_planet=r_planet,
# )